# Redes neuronales

In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# cargar data frame con archivo colab notebooks/Sindicados 2020.xlsx
dfSindicados = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DataSets/Sindicados 2020.xlsx')

In [ ]:
# cargar data frame con archivo colab botebooks notebooks/Agraviados 2020.xlsx
dfAgraviados = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DataSets/Agraviados 2020.xlsx')

In [ ]:
# mostrar primeras filas del conjunto de datos
dfSindicados.head()


,núm_corre,año_denuncia,mes_denuncia,día_denuncia,día_sem_denuncia,año_hecho,mes_hecho,día_hecho,día_sem_hecho,depto_ocu_hecho,...,zona_ocu_hecho,edad_sind,g_edad_60ymás,g_edad_80ymás,g_edad_de0_80ymás,est_conyugal,sexo_sindicados,delito_cod,mayor_edad,tercera_edad
38481,240006,2020,2,13,4,9999,99,99,9,1,...,21,999,12,16,18,9,1,231,1,1
410019,93682,2020,2,21,5,2019,8,23,5,12,...,99,999,12,16,18,9,1,11,1,1
50990,79795,2020,7,29,3,2020,7,29,3,10,...,99,35,6,6,8,1,1,227,1,0
176183,89705,2020,2,27,4,2020,2,25,2,1,...,99,999,12,16,18,9,9,311,1,1
357823,22023,2020,10,27,2,2020,10,4,7,14,...,99,999,12,16,18,9,2,901,1,1


In [ ]:
# mostrar primeras filas
dfAgraviados.head()

,núm_corre,año_denuncia,mes_denuncia,día_denuncia,día_sem_denuncia,año_hecho,mes_hecho,dia_sem_hecho,día_hecho,depto_ocu_hecho,mupio_ocu_hecho,zona_ocu_hecho,edad_agrav,g_edad_60ymás,g_edad_80ymás,g_edad_de0_80ymás,est_conyugal,sexo_agraviados,delito_cod
0,1,2020,3,3,2,2019,9,7,1,18,1801,99,999,12,16,18,2,1,177
1,2,2020,3,6,5,2018,1,1,1,21,2102,99,53,9,9,11,2,1,235
2,3,2020,5,13,3,2019,1,2,1,5,501,99,56,10,10,12,1,2,177
3,4,2020,5,20,3,2019,1,2,1,1,101,99,999,12,16,18,1,2,254
4,5,2020,5,20,3,2019,1,2,1,1,101,99,25,4,4,6,1,2,200


In [ ]:
# inyectar aleatoriedad
dfSindicados = shuffle(dfSindicados)
dfAgraviados = shuffle(dfAgraviados)

### Preparar sindicados

In [ ]:
print(dfSindicados.shape)

# Sindicados
# Crear columna para saber si es mayor de edad o no
dfSindicados['mayor_edad'] = np.where(dfSindicados['edad_sind'] >= 18, 1, 0)

# Crear columna para saber si es de la tercera edad o no
dfSindicados['tercera_edad'] = np.where(dfSindicados['edad_sind'] >= 65, 1, 0)

# Filtrar delitos de la categoría robo
dfSindicados_ = dfSindicados[(dfSindicados['delito_cod'] >= 183)  & (dfSindicados['delito_cod'] <= 190)]
dfSindicados_.shape

x = dfSindicados_[['delito_cod','est_conyugal','depto_ocu_hecho','día_sem_denuncia']]
y = dfSindicados_['sexo_sindicados']

x_2 = dfSindicados[['delito_cod','sexo_sindicados','depto_ocu_hecho','día_sem_denuncia']]
y_2 = dfSindicados['est_conyugal']

x_4 = dfSindicados[['delito_cod','sexo_sindicados','depto_ocu_hecho','día_sem_denuncia','zona_ocu_hecho', 'día_sem_hecho']]
y_4 = dfSindicados['tercera_edad']


(427668, 21)


### Preparar agraviados

In [ ]:
print(dfAgraviados.shape)

# Agraviados
# crear columna para saber si es mayor de edad o no
dfAgraviados['mayor_edad'] = np.where(dfAgraviados['edad_agrav'] >= 18, 1, 0)

# crear columna para saber si es de la tercera edad o no
dfAgraviados['tercera_edad'] = np.where(dfAgraviados['edad_agrav'] >= 65, 1, 0)

# filtrar delitos de la categoría robo
dfAgraviados_ = dfAgraviados[(dfAgraviados['delito_cod'] >= 183)  & (dfAgraviados['delito_cod'] <= 190)]

# filtrar solo los elementos en los que delito_cod sea mayor 183 y menor a 190, robos menores
dfSindicados_ = dfSindicados_[(dfSindicados_['depto_ocu_hecho'] == 1) ]
dfAgraviados_.shape

x_1 = dfAgraviados_[['delito_cod','est_conyugal','depto_ocu_hecho','día_sem_denuncia']]
y_1 = dfAgraviados_['sexo_agraviados']

x_3 = dfAgraviados_[['delito_cod','sexo_agraviados','depto_ocu_hecho','est_conyugal', 'zona_ocu_hecho','dia_sem_hecho']]
y_3 = dfAgraviados_['tercera_edad']

(365031, 21)


## Crear modelo de red neuronal artificial

In [ ]:

X_train, X_test, y_train, y_test  = train_test_split(x, y, test_size = 0.2, random_state = 42)

# crear modelo
model = Sequential()
model.add(Dense(16, input_dim = 4, activation = 'relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# compilar modelo
model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [ ]:
# entrenar y testear
model.fit(X_train, y_train, epochs = 10, batch_size = 64, validation_data=(X_test, y_test))

Epoch 1/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3825 - loss: 0.2318 - val_accuracy: 0.3931 - val_loss: -1.1394
Epoch 2/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3943 - loss: -1.5827 - val_accuracy: 0.3931 - val_loss: -2.9317
Epoch 3/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3920 - loss: -3.3692 - val_accuracy: 0.3931 - val_loss: -4.6818
Epoch 4/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3929 - loss: -5.0913 - val_accuracy: 0.3931 - val_loss: -6.4002
Epoch 5/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3916 - loss: -6.8196 - val_accuracy: 0.3931 - val_loss: -8.0950
Epoch 6/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3937 - loss: -8.4625 - val_accuracy: 0.3931 - val_loss: -9.7742
Epoch 7/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3896 - loss: -10.2472 - val_accuracy: 0.3931 - val_loss: -11.4429
Epoch 8/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3900 - loss: -11.8652 -

In [ ]:
# obtener presición del modelo
loss, acc = model.evaluate(X_test, y_test)
print(acc*100)

195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4045 - loss: -16.1104
39.30524289608002


In [ ]:
# [['delito_cod','est_conyugal','depto_ocu_hecho','día_sem_denuncia']]

# Predicción de casos nuevos
vuelo_nuevo_01 = np.array([[183, 3, 5, 2]])

# predecir con el modelo
prediccion = model.predict(vuelo_nuevo_01)
print(prediccion)

# predecir otro valor
vuelo_nuevo_02 = np.array([[185, 2, 1, 3]])
prediccion = model.predict(vuelo_nuevo_02)
print(prediccion)

# predecir otro valor
vuelo_nuevo_03 = np.array([[189, 4, 1, 1]])
prediccion = model.predict(vuelo_nuevo_03)
print(prediccion)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
[[0.97811425]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.97811425]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.97811425]]


## Modelo 2, predicción de sexo del agraviado

In [ ]:
# Crear separación de datos para ejecución dle algoritmo

#X_train, X_test, y_train, y_test  = train_test_split(x, y, test_size = 0.2, random_state = 42)
X_train, X_test, y_train, y_test  = train_test_split(x_1, y_1, test_size = 0.2, random_state = 42)

# crear modelo
model = Sequential()
model.add(Dense(16, input_dim = 4, activation = 'relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# compilar modelo
model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics =['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# entrenar y testear
model.fit(X_train, y_train, epochs = 7, batch_size = 64, validation_data=(X_test, y_test))

Epoch 1/7
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4005 - loss: 3.6243 - val_accuracy: 0.6527 - val_loss: 0.3910
Epoch 2/7
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6497 - loss: 0.3168 - val_accuracy: 0.6530 - val_loss: 0.1006
Epoch 3/7
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6461 - loss: 0.0395 - val_accuracy: 0.6530 - val_loss: -0.1647
Epoch 4/7
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6463 - loss: -0.2264 - val_accuracy: 0.6530 - val_loss: -0.4072
Epoch 5/7
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6562 - loss: -0.4511 - val_accuracy: 0.6530 - val_loss: -0.6301
Epoch 6/7
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6464 - loss: -0.6693 - val_accuracy: 0.6530 - val_loss: -0.8400
Epoch 7/7
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6437 - loss: -0.9037 - val_accuracy: 0.6530 - val_loss: -1.0377


In [ ]:
# obtener presición del modelo
loss, acc = model.evaluate(X_test, y_test)
print(acc*100)

165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6561 - loss: -1.0149
65.3030276298523


In [ ]:
# [['delito_cod','est_conyugal','depto_ocu_hecho','día_sem_denuncia']]

# Predicción de casos nuevos
vuelo_nuevo_01 = np.array([[183, 3, 5, 2]])

# predecir con el modelo
prediccion = model.predict(vuelo_nuevo_01)
print(prediccion)

# predecir otro valor
vuelo_nuevo_02 = np.array([[185, 2, 1, 3]])
prediccion = model.predict(vuelo_nuevo_02)
print(prediccion)

# predecir otro valor
vuelo_nuevo_03 = np.array([[189, 4, 1, 1]])
prediccion = model.predict(vuelo_nuevo_03)
print(prediccion)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
[[0.8833973]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.8833973]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.8833973]]


## Modelo 03 - Predicción tercera edad agraviados

In [ ]:
# Crear separación de datos para ejecución dle algoritmo

X_train, X_test, y_train, y_test  = train_test_split(x_3, y_3, test_size = 0.2, random_state = 42)

# crear modelo
model = Sequential()
model.add(Dense(16, input_dim = 6, activation = 'relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# compilar modelo
model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics =['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# entrenar y testear
model.fit(X_train, y_train, epochs = 8, batch_size = 64, validation_data=(X_test, y_test))

Epoch 1/8
330/330 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6130 - loss: 0.7441 - val_accuracy: 0.7017 - val_loss: 0.6356
Epoch 2/8
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6890 - loss: 0.6395 - val_accuracy: 0.7057 - val_loss: 0.6139
Epoch 3/8
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7074 - loss: 0.6130 - val_accuracy: 0.7045 - val_loss: 0.6056
Epoch 4/8
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7243 - loss: 0.5935 - val_accuracy: 0.7661 - val_loss: 0.5543
Epoch 5/8
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7558 - loss: 0.5593 - val_accuracy: 0.7669 - val_loss: 0.5464
Epoch 6/8
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7544 - loss: 0.5547 - val_accuracy: 0.7657 - val_loss: 0.5400
Epoch 7/8
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7605 - loss: 0.5467 - val_accuracy: 0.7534 - val_loss: 0.5444
Epoch 8/8
330/330 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7570 - loss: 0.5470 - val_accuracy: 0.7657 - v

In [ ]:
# obtener presición del modelo
loss, acc = model.evaluate(X_test, y_test)
print(acc*100)

165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7558 - loss: 0.5482
76.57197117805481


In [ ]:
# Predicción de casos nuevos
vuelo_nuevo_01 = np.array([[188, 1, 1, 4, 7, 3]])

# predecir con el modelo
prediccion = model.predict(vuelo_nuevo_01)
print(prediccion)

# predecir otro valor
vuelo_nuevo_02 = np.array([[185, 2, 5, 1, 8, 1]])
prediccion = model.predict(vuelo_nuevo_02)
print(prediccion)

# predecir otro valor
vuelo_nuevo_03 = np.array([[183, 2, 3, 1, 15, 3]])
prediccion = model.predict(vuelo_nuevo_03)
print(prediccion)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
[[0.24319075]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.19832045]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[0.20167246]]


## Modelo 04 - Predicción de tercera edad para sindicados

In [ ]:
# Crear separación de datos para ejecución dle algoritmo

X_train, X_test, y_train, y_test  = train_test_split(x_4, y_4, test_size = 0.2, random_state = 42)

# crear modelo
model = Sequential()
model.add(Dense(16, input_dim = 6, activation = 'relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# compilar modelo
model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics =['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# entrenar y testear
historial = model.fit(X_train, y_train, epochs = 8, batch_size = 64, validation_data=(X_test, y_test))

Epoch 1/8
5346/5346 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.7981 - loss: 0.5926 - val_accuracy: 0.8057 - val_loss: 0.4388
Epoch 2/8
5346/5346 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.8036 - loss: 0.4419 - val_accuracy: 0.8057 - val_loss: 0.4388
Epoch 3/8
5346/5346 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.8029 - loss: 0.4420 - val_accuracy: 0.8057 - val_loss: 0.4377
Epoch 4/8
5346/5346 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.8033 - loss: 0.4412 - val_accuracy: 0.8057 - val_loss: 0.4378
Epoch 5/8
5346/5346 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.8041 - loss: 0.4397 - val_accuracy: 0.8057 - val_loss: 0.4374
Epoch 6/8
5346/5346 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.8033 - loss: 0.4405 - val_accuracy: 0.8057 - val_loss: 0.4384
Epoch 7/8
5346/5346 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.8031 - loss: 0.4406 - val_accuracy: 0.8057 - val_loss: 0.4371
Epoch 8/8
5346/5346 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.8041 - loss: 0.4394 - 

In [ ]:
# obtener presición del modelo
loss, acc = model.evaluate(X_test, y_test)
print(acc*100)

2673/2673 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8007 - loss: 0.4426
80.22657632827759


In [ ]:
# predicción de un valor nuevo
# x_4 = dfSindicados[['delito_cod','sexo_sindicados','depto_ocu_hecho','día_sem_denuncia','zona_ocu_hecho', 'día_sem_hecho']]
# crear objeto nuevo
vuelo_nuevo_01 = np.array([[188, 1, 1, 4, 7, 3]])

# predecir con el modelo
prediccion = model.predict(vuelo_nuevo_01)
print(prediccion)

# predecir otro valor
vuelo_nuevo_02 = np.array([[185, 2, 5, 1, 8, 1]])
prediccion = model.predict(vuelo_nuevo_02)
print(prediccion)

# predecir otro valor
vuelo_nuevo_03 = np.array([[183, 2, 3, 1, 15, 3]])
prediccion = model.predict(vuelo_nuevo_03)
print(prediccion)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.7225446]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[[0.83651894]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[[0.8393722]]


In [ ]:
# Graficar la pérdida
plt.plot(historial.history['loss'])
plt.plot(historial.history['val_loss'])
plt.title('Pérdida del modelo')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
plt.show()

# Graficar la precisión
plt.plot(historial.history['accuracy'])
plt.plot(historial.history['val_accuracy'])
plt.title('Precisión del modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='lower right')
plt.show()

NameError: name 'historial' is not defined